# nb2fasthtml

> Create FastHTML from a NB

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from pathlib import Path
import json, yaml
from execnb.nbio import *
from execnb.shell import render_outputs
from typing import Callable
from functools import partial

In [ ]:
from IPython.display import HTML,Markdown

In [ ]:
#| export
hdrs = (MarkdownJS(), HighlightJS(langs=['python', 'javascript', 'html', 'css']))

In [ ]:
nb_dir = Path('../example_nbs/')
xt_nb = read_nb(nb_dir/'explaining_xt_components.ipynb')
cells = xt_nb.cells

In [ ]:
#| export
def strip_list(l, val='\n'):
    start, end = 0, len(l)
    while start < end and l[start] == val: start += 1
    while end > start and l[end - 1] == val: end -= 1
    return l[start:end]

## Render Cells

In [ ]:
#| export
def get_nb_lang(nb):
    "Get the language of `nb`"
    return nb['metadata']['kernelspec']['language']

In [ ]:
get_nb_lang(xt_nb)

'python'

In [ ]:
#| export
def render_md(c, container=Div):
    "Default rendering function; adds class 'marked' for use with highlight-js"
    return container(c, cls="marked")

In [ ]:
md_cell = first(o for o in cells if o.cell_type=='markdown')
render_md(md_cell.source)

```html
<div class="marked"># **ft** Components</div>

```

In [ ]:
from mistletoe import markdown, HTMLRenderer
from mistletoe.contrib.pygments_renderer import PygmentsRenderer

In [ ]:
def _render_md(c, **kw):
    "Non exported version for display in nb"
    return markdown(c, renderer=PygmentsRenderer)

In [ ]:
#| export
def render_code_source(cell, lang='python', render_md=render_md):
    if not cell.source: return ''
    return render_md(f'''\n```{lang}\n{cell.source}\n```\n''')

In [ ]:
code_cell = first(o for o in cells if o.cell_type=='code')
HTML(render_code_source(code_cell, render_md=_render_md))

In [ ]:
out_cell = first(o for o in cells if o.get('outputs'))
out_cell.outputs

[{'data': {'text/markdown': ['```xml\n',
    '<div class="go">\n',
    '  <h1>FastHTML APP</h1>\n',
    '  <p>Let&#x27;s do this</p>\n',
    '</div>\n',
    '\n',
    '```'],
   'text/plain': ["['div',\n",
    ' ([\'h1\', (\'FastHTML APP\',), {}], [\'p\', ("Let\'s do this",), {}]),\n',
    " {'class': 'go'}]"]},
  'execution_count': None,
  'metadata': {},
  'output_type': 'execute_result'}]

In [ ]:
def _join(s): return ''.join(s).strip()

In [ ]:
md_out = _join(out_cell.outputs[0]['data']['text/markdown'])
print(md_out)

```xml
<div class="go">
  <h1>FastHTML APP</h1>
  <p>Let&#x27;s do this</p>
</div>

```


In [ ]:
#| export
def render_code_output(cell, lang='python', pygments=False, wrapper=Footer):
    if not cell.outputs: return ''
    res = render_outputs(cell.outputs, pygments=pygments)
    if res: return wrapper(NotStr(res))

In [ ]:
HTML(render_outputs(out_cell.outputs, pygments=True))

## Frontmatter

In [ ]:
#| export
_RE_FM_BASE=r'''^---\s*
(.*?\S+.*?)
---\s*'''

In [ ]:
#| export
_re_fm_nb = re.compile(_RE_FM_BASE+'$', flags=re.DOTALL)
_re_fm_md = re.compile(_RE_FM_BASE, flags=re.DOTALL)

In [ ]:
#| export
def _fm2dict(s:str, nb=True):
    "Load YAML frontmatter into a `dict`"
    re_fm = _re_fm_nb if nb else _re_fm_md
    match = re_fm.search(s.strip())
    return yaml.safe_load(match.group(1)) if match else {}

In [ ]:
#| export
def _md2dict(s:str):
    "Convert H1 formatted markdown cell to frontmatter dict"
    if '#' not in s: return {}
    m = re.search(r'^#\s+(\S.*?)\s*$', s, flags=re.MULTILINE)
    if not m: return {}
    res = {'title': m.group(1)}
    m = re.search(r'^>\s+(\S.*?)\s*$', s, flags=re.MULTILINE)
    if m: res['description'] = m.group(1)
    r = re.findall(r'^-\s+(\S.*:.*\S)\s*$', s, flags=re.MULTILINE)
    if r:
        try: res.update(yaml.safe_load('\n'.join(r)))
        except Exception as e: warn(f'Failed to create YAML dict for:\n{r}\n\n{e}\n')
    return res

In [ ]:
#| export
def get_frontmatter(source,     # metatadata source (jupyter cell or md content)
                    nb_file=True,    # Is jupyter nb or qmd file
                    md_fm=False # md or raw style frontmatter
                   ):
    if not nb_file: return _fm2dict(source)
    if md_fm:       return _md2dict(source.source)
    return _fm2dict(source.source, nb_file)    

get_frontmatter_raw = partial(get_frontmatter, md_fm=False)
get_frontmatter_md =  partial(get_frontmatter, md_fm=True)

In [ ]:
nb = read_nb(nb_dir/'by_example.ipynb')
fm = get_frontmatter(nb.cells[0], md_fm=True)
fm

{'title': 'FastHTML By Example',
 'description': 'An introduction to FastHTML from the ground up, with four complete examples',
 'order': 2}

In [ ]:
#| export
def render_frontmatter(fm):
    desc = P(fm['description']) if 'description' in fm else ()
    return Div(cls='frontmatter')(
        H1(fm['title']), desc
    )

In [ ]:
show(render_frontmatter(fm))

## Render NB

In [ ]:
#| export
def render_nb(fpath, # Path to Jupyter Notebook
              wrapper=Main, #Wraps entire rendered NB, default is for pico
              cls='container', # cls to be passed to wrapper, default is for pico
              md_cell_wrapper=Div, # Wraps markdown cell
              md_fn=render_md, # md -> rendered html
              code_cell_wrapper=Card, # Wraps Source Code (body) + Outputs (footer)
              cd_fn=render_code_source, # code cell -> code source rendered html
              out_fn=render_code_output, # code cell -> code output rendered html
              get_fm=get_frontmatter_md, # How to read frontmatter cell
              fm_fn:None|Callable=render_frontmatter, # Frontmatter -> FT components
              **kwargs # Passed to wrapper
             ): 
    nb = read_nb(fpath)
    res, content_start_idx = [], 0
    if fm_fn: 
        content_start_idx = 1
        fm = get_fm(nb.cells[0])
        res.append(fm_fn(fm))
    for cell in nb.cells[content_start_idx:]:
        if   cell['cell_type']=='code'    : res.append(code_cell_wrapper(cd_fn(cell), out_fn(cell)))
        elif cell['cell_type']=='markdown': res.append(md_cell_wrapper(md_fn(cell.source)))
    return wrapper(cls=cls, **kwargs)(*res)

In [ ]:
if IN_NOTEBOOK: from fasthtml.jupyter import *

In [ ]:
if IN_NOTEBOOK:
    app, rt = fast_app(hdrs=hdrs)
    server = JupyUvi(app)

    @app.get
    def index(): return rendered_nb

In [ ]:
rendered_nb = render_nb(nb_dir/'by_example.ipynb')

In [ ]:
print(to_xml(rendered_nb)[:100])

<main class="container">  <div class="frontmatter">
    <h1>FastHTML By Example</h1>
    <p>An intro


In [ ]:
# HTMX(link=True, iframe=False)

In [ ]:
if IN_NOTEBOOK: server.stop()

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()